# Multi-Agent AI Demo: From News to Investment Suggestion

In this notebook we simulate a simple multi-agent AI pipeline:

1. **StockDataAgent** – fetches S&P 500 price data.
2. **NewsAgent** – retrieves or accepts a news article.
3. **SummaryAgent** – summarizes the article.
4. **SentimentAgent** – analyzes the sentiment of the summary.
5. **DecisionAgent** – produces a *toy* investment suggestion based on the sentiment and recent market move.

This is a **teaching example only**; it is **not** real financial advice.


In [ ]:
from ai_agents import (
    StockDataAgent,
    NewsAgent,
    SummaryAgent,
    SentimentAgent,
    DecisionAgent,
)


## Step 1 – Fetch recent S&P 500 data

We first use `StockDataAgent` to retrieve recent price data for the S&P 500 index
(symbol `^GSPC`) and compute a simple recent percentage return. This gives our
later decision agent some basic market context.


In [ ]:
stock_agent = StockDataAgent()

# Fetch the last year of daily data for S&P 500
price_history = stock_agent.get_index_history(symbol="^GSPC", period="1y", interval="1d")

recent_return = stock_agent.compute_recent_return(price_history, lookback_days=5)

print("Last 5-day approx. return:", f"{recent_return*100:.2f}%" if recent_return is not None else "N/A")

price_history.tail()


## Step 2 – Get a news article

Now we use `NewsAgent` to obtain an article. There are two options:

1. Provide a URL to a news article (basic HTML scraping).
2. Manually paste the article text.

For workshop purposes, you can show both approaches. Below we use **manual input**
so that the example is stable and reproducible.


In [ ]:
news_agent = NewsAgent()

# Option A: use a URL (uncomment and set a real URL)
# url = "https://example.com/some-news-article"
# article = news_agent.fetch_article_from_url(url, source_name="Example News")

# Option B: manual input (recommended for a live demo)
article_title = "Central bank signals possible rate cuts as inflation cools"
article_content = """
Central bank officials indicated on Thursday that interest rates may be cut later
this year if inflation continues to ease. In a press conference, the governor
highlighted that recent data show a slowdown in price increases and more stable
labor markets. However, policymakers warned that they remain cautious and will
adjust their stance if new risks emerge in the global economy.
"""

article = news_agent.create_article_from_manual_input(
    title=article_title,
    content=article_content,
    source_name="Demo News Outlet",
)

article


## Step 3 – Summarize the article

We now pass the article to the `SummaryAgent`. In this simplified version,
the agent extracts the first few sentences as a summary.

In a more advanced setup, this agent could call a large language model (LLM)
for higher-quality abstractive summarization.


In [ ]:
summary_agent = SummaryAgent(max_sentences=3)

summary_result = summary_agent.summarize_article(article)

print("=== Article title ===")
print(article.title)
print("\n=== Summary ===")
print(summary_result.summary)
print("\nSummary model:", summary_result.model_name)


## Step 4 – Analyze sentiment of the summary

Next, the `SentimentAgent` takes the summary and classifies it as **POSITIVE**,
**NEGATIVE**, or **NEUTRAL**, along with a confidence score between 0 and 1.

This agent encapsulates a pre-trained sentiment model behind a simple interface.


In [ ]:
sentiment_agent = SentimentAgent()

sentiment_result = sentiment_agent.analyze_text(summary_result.summary)

print("Sentiment label:", sentiment_result.label)
print("Sentiment score:", f"{sentiment_result.score:.3f}")
print("Sentiment model:", sentiment_result.model_name)


## Step 5 – Generate a toy investment suggestion

Finally, the `DecisionAgent` combines:

- the sentiment result, and  
- the recent market return for the S&P 500,

to produce a **toy investment suggestion**.

This is a simple rule-based mapping from sentiment to a human-readable action.
It illustrates how agents can be chained together into a decision-making pipeline.


In [ ]:
decision_agent = DecisionAgent()

recommendation = decision_agent.make_recommendation(
    sentiment=sentiment_result,
    recent_return=recent_return,
)

print("=== Toy investment suggestion (education only) ===")
print("Recommendation:", recommendation.recommendation)
print("Confidence:", f"{recommendation.confidence:.2f}")
print("Rationale:\n", recommendation.rationale)


## Wrap-up

In this notebook we have:

1. Collected recent S&P 500 data.
2. Retrieved or manually provided a news article.
3. Summarized the article.
4. Analyzed the sentiment of the summary.
5. Translated sentiment into a toy investment suggestion.

Each step was handled by a **separate agent class**, showing how a multi-agent
AI pipeline can be composed of specialized components that pass information
along a chain.

For your workshop, you can:
- Step through each cell and discuss what each agent does.
- Show or modify the internals of each module.
- Experiment with different articles and see how the sentiment/decision change.
